In [ ]:
!pip install ultralytics==8.0.20
import ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from tqdm import tqdm
import PIL.Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os.path
from natsort import natsorted
import glob

In [ ]:
!yolo task=segment mode=predict model='/content/drive/MyDrive/ETVProject/ETV_Project/weights/best.pt' conf=0.86 source='/content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images' save=True boxes=False

2023-10-03 10:45:24.926470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 10:45:25.804895: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782309 parameters, 0 gradients, 42.5 GFLOPs
image 1/319 /content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images/100001.png: 192x224 75.1ms
image 2/319 /content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images/100002.png: 192x224 1 Choroid Plexus, 7.7ms
image 3/319 /content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images/100003.png: 192x224 1 Choroid Plexus, 7.5ms
image 4/319 

In [ ]:
model = YOLO('/content/drive/MyDrive/ETVProject/ETV_Project/weights/best.pt')
source='/content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images'
imlist = natsorted(os.listdir('/content/drive/MyDrive/ETVProject/ETV_Project/chiari_etv_2fps/images'))

In [ ]:
'''
for i in tqdm(imlist):
  im=PIL.Image.open(i)
  if im.size!=(224, 224):
    im=im.resize((224, 224))
    im=im.save(i)
'''

'\nfor i in tqdm(imlist):\n  im=PIL.Image.open(i)\n  if im.size!=(224, 224):\n    im=im.resize((224, 224))\n    im=im.save(i)\n'

In [ ]:
results=[]
for im in tqdm(imlist):
  if '.png' in im:
    #print(source+'/'+im)
    results.append(model(source+'/'+im))

  0%|          | 0/319 [00:00<?, ?it/s]Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11782309 parameters, 0 gradients, 42.5 GFLOPs
100%|██████████| 319/319 [00:15<00:00, 21.24it/s]


In [ ]:
fps = 2
limit = fps * 3
count=0
phase1_idx = phase2_idx = phase3_idx = 0
flag1 = flag2 = flag3 = False
phase1_count = phase2_count = phase3_count = 0
for i in tqdm(range(len(results))):
  print(i)
  print(results[i][0].boxes.cls)
  if flag1==False and (1.0 in results[i][0].boxes.cls or 2.0 in results[i][0].boxes.cls) and (results[i][0].boxes.conf[0] > 0.83 or (len(results[i][0].boxes.conf)>1 and results[i][0].boxes.conf[1] > 0.83)):
    phase1_count = phase1_count + 1
    if phase1_count >= limit:
      flag1 = True
      print(flag1)
      phase1_idx = count
  elif flag1==False and flag2==False and 1.0 not in results[i][0].boxes.cls and 2.0 not in results[i][0].boxes.cls:
    phase1_count = 0

  if flag1==True and flag2==False and 1.0 not in results[i][0].boxes.cls and 2.0 not in results[i][0].boxes.cls:
    phase2_count = phase2_count + 1
    if phase2_count >= limit:
      flag2 = True
      phase2_idx = count
  elif flag1==True and flag2==False and 1.0 in results[i][0].boxes.cls and 2.0 in results[i][0].boxes.cls:
    phase2_count = 0

  if flag1==True and flag2==True and flag3==False and (3.0 in results[i][0].boxes.cls and 5.0 in results[i][0].boxes.cls):
    # and results[i][0].boxes.conf[0] > 0.86 or (len(results[i][0].boxes.conf)>1 and results[i][0].boxes.conf[1] > 0.86)
    phase3_count = phase3_count + 1
    if phase3_count >= limit:
      flag3 = True
      phase3_idx = count
  elif flag3==False and 3.0 not in results[i][0].boxes.cls and 5.0 not in results[i][0].boxes.cls:
    phase3_count = 0
  count+=1

print()
print(phase1_idx, phase2_idx, phase3_idx)


 32%|███▏      | 103/319 [00:00<00:00, 520.08it/s]

0
tensor([1., 2.], device='cuda:0')
1
tensor([1., 2.], device='cuda:0')
2
tensor([1., 2.], device='cuda:0')
3
tensor([1., 2.], device='cuda:0')
4
tensor([1., 2.], device='cuda:0')
5
tensor([2., 1.], device='cuda:0')
True
6
tensor([1., 2.], device='cuda:0')
7
tensor([1., 2.], device='cuda:0')
8
tensor([1., 2.], device='cuda:0')
9
tensor([1., 2.], device='cuda:0')
10
tensor([1., 2.], device='cuda:0')
11
tensor([1., 2.], device='cuda:0')
12
tensor([1., 2.], device='cuda:0')
13
tensor([1., 2.], device='cuda:0')
14
tensor([1., 2.], device='cuda:0')
15
tensor([1., 2.], device='cuda:0')
16
tensor([1., 2.], device='cuda:0')
17
tensor([1., 2.], device='cuda:0')
18
tensor([1., 2.], device='cuda:0')
19
tensor([1., 2.], device='cuda:0')
20
tensor([1., 2.], device='cuda:0')
21
tensor([1., 2.], device='cuda:0')
22
tensor([1., 2.], device='cuda:0')
23
tensor([1., 2.], device='cuda:0')
24
tensor([1., 2.], device='cuda:0')
25
tensor([1., 2.], device='cuda:0')
26
tensor([2., 1.], device='cuda:0')
27
ten

 68%|██████▊   | 217/319 [00:00<00:00, 548.03it/s]

103
tensor([5., 3., 6., 4.], device='cuda:0')
104
tensor([5., 3., 6., 4.], device='cuda:0')
105
tensor([5., 3., 6., 4.], device='cuda:0')
106
tensor([5., 3., 6., 4.], device='cuda:0')
107
tensor([5., 3., 6., 4.], device='cuda:0')
108
tensor([5., 3., 6., 4.], device='cuda:0')
109
tensor([5., 3., 4., 6.], device='cuda:0')
110
tensor([5., 3., 4., 6.], device='cuda:0')
111
tensor([5., 3., 4., 6.], device='cuda:0')
112
tensor([5., 3., 6., 4.], device='cuda:0')
113
tensor([5., 3., 4., 6.], device='cuda:0')
114
tensor([5., 3., 4., 6.], device='cuda:0')
115
tensor([5., 3., 4., 6.], device='cuda:0')
116
tensor([5., 3., 4., 6.], device='cuda:0')
117
tensor([5., 4., 3., 6.], device='cuda:0')
118
tensor([5., 3., 4., 6.], device='cuda:0')
119
tensor([5., 3., 4., 6.], device='cuda:0')
120
tensor([3., 4., 5., 6.], device='cuda:0')
121
tensor([3., 6., 4., 5.], device='cuda:0')
122
tensor([4., 3., 5., 6.], device='cuda:0')
123
tensor([4., 3., 5., 6.], device='cuda:0')
124
tensor([3., 4., 6., 5.], devic

100%|██████████| 319/319 [00:00<00:00, 556.29it/s]

217
tensor([4., 6., 5., 3.], device='cuda:0')
218
tensor([5., 4., 6., 3.], device='cuda:0')
219
tensor([5., 6., 3., 4.], device='cuda:0')
220
tensor([5., 4., 6., 3.], device='cuda:0')
221
tensor([5., 4., 6., 3.], device='cuda:0')
222
tensor([5., 6., 3., 4.], device='cuda:0')
223
tensor([5., 6., 3., 4.], device='cuda:0')
224
tensor([5., 3., 6., 4., 4., 4.], device='cuda:0')
225
tensor([5., 6., 4., 3.], device='cuda:0')
226
tensor([5., 6., 4., 3.], device='cuda:0')
227
tensor([5., 6., 4., 3.], device='cuda:0')
228
tensor([6., 5., 4., 3.], device='cuda:0')
229
tensor([6., 5., 3., 4.], device='cuda:0')
230
tensor([6., 5., 3., 4., 4.], device='cuda:0')
231
tensor([6., 3., 5., 4., 4.], device='cuda:0')
232
tensor([6., 3., 5., 4.], device='cuda:0')
233
tensor([3., 6., 5., 4.], device='cuda:0')
234
tensor([3., 6., 5., 4.], device='cuda:0')
235
tensor([3., 6., 5., 4.], device='cuda:0')
236
tensor([3., 6., 5., 4., 4.], device='cuda:0')
237
tensor([3., 6., 5., 4., 4.], device='cuda:0')
238
tensor

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16003, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 16003 (delta 21), reused 20 (delta 12), pack-reused 15967
Receiving objects: 100% (16003/16003), 14.66 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (10987/10987), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.8/615.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 36.7 MB/s eta

In [ ]:
!python detect.py --weights /content/drive/MyDrive/ETVProject/ETV_Project/yolov5best.pt --img 224 --conf 0.5 --source /content/runs/segment/predict/

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'.
detect: weights=['/content/drive/MyDrive/ETVProject/ETV_Project/yolov5best.pt'], source=/content/runs/segment/predict/, data=data/coco128.yaml, imgsz=[224, 224], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-225-gbb9706e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 157 lay

In [ ]:
model = torch.hub.load('.','custom', path='/content/drive/MyDrive/ETVProject/ETV_Project/yolov5best.pt',source='local')

YOLOv5 🚀 v7.0-225-gbb9706e Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
source = '/content/runs/segment/predict/'
imlist = natsorted(os.listdir('/content/runs/segment/predict/'))
for i in tqdm(range(len(imlist))):
  imlist[i]=source+imlist[i]

100%|██████████| 319/319 [00:00<00:00, 1217454.94it/s]


In [ ]:
results = model(imlist, size=224)

In [ ]:
results.print()

image 1/319: 576x720 (no detections)
image 2/319: 576x720 (no detections)
image 3/319: 576x720 (no detections)
image 4/319: 576x720 (no detections)
image 5/319: 576x720 (no detections)
image 6/319: 576x720 (no detections)
image 7/319: 576x720 (no detections)
image 8/319: 576x720 1 B
image 9/319: 576x720 (no detections)
image 10/319: 576x720 (no detections)
image 11/319: 576x720 (no detections)
image 12/319: 576x720 (no detections)
image 13/319: 576x720 (no detections)
image 14/319: 576x720 (no detections)
image 15/319: 576x720 (no detections)
image 16/319: 576x720 (no detections)
image 17/319: 576x720 (no detections)
image 18/319: 576x720 (no detections)
image 19/319: 576x720 (no detections)
image 20/319: 576x720 (no detections)
image 21/319: 576x720 (no detections)
image 22/319: 576x720 (no detections)
image 23/319: 576x720 (no detections)
image 24/319: 576x720 (no detections)
image 25/319: 576x720 (no detections)
image 26/319: 576x720 (no detections)
image 27/319: 576x720 (no detecti

In [ ]:
import numpy as np


In [ ]:
fps = 2
limit = fps * 3
count = 0
phase4_count = phase5_count = phase4_idx = phase5_idx = phase6_idx = phase6_count = 0
flag4 = flag5 = flag6 = False

for i in tqdm(range(len(results))):
  if i <= phase3_idx:
    count+=1
    continue

  arr = results.xyxy[i]

  if flag4 == False and flag5 == False and 1 in arr:
    phase4_count = phase4_count + 1
    if phase4_count>=limit:
      flag4 = True
      phase4_idx = count
  elif flag4 == False and flag5 == False and 1 not in arr:
    phase4_count = 0

  if flag4 == True and flag5 == False and 0 in arr:
    phase5_count = phase5_count + 1
    if phase5_count>=limit:
      flag5 = True
      phase5_idx = count
  elif flag4 == True and flag5 == False and 0 not in arr:
    phase5_count = 0

  if flag4 == True and flag5 == True and flag6 == False and 0 not in arr:
    phase6_count +=1
    if phase6_count>=limit:
      flag6 = True
      phase6_idx = count
  elif flag4 == True and flag5 == True and flag6 == False and 0 in arr:
    phase6_count = 0
  count+=1


print(phase4_idx, phase5_idx, phase6_idx)

100%|██████████| 319/319 [00:00<00:00, 20593.54it/s]

88 189 242


In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [ ]:
count = 0
imlist = natsorted(os.listdir('/content/yolov5/runs/detect/exp/'))
for f in tqdm(imlist):
  if '.png' in f:
    filename = "/content/yolov5/runs/detect/exp/"+f
    img = Image.open(filename)
    I1 = ImageDraw.Draw(img)

    if count >=phase6_idx:
      I1.text((10, 10), "5 Phases Complete", fill =(255, 0, 0), size=75)
    elif count >= phase5_idx:
      I1.text((10, 10), "Dilation", fill =(255, 0, 0), size=75)
    elif count >= phase4_idx:
      I1.text((10, 10), "Puncturing", fill =(255, 0, 0), size=75)
    elif count >= phase3_idx:
      I1.text((10, 10), "Third Ventricle", fill =(255, 0, 0), size=75)
    elif count >= phase2_idx:
      I1.text((10, 10), "Entering Third Ventricle", fill =(255, 0, 0), size=75)
    elif count >= phase1_idx:
      I1.text((10, 10), "Lateral Ventricle", fill =(255, 0, 0), size=75)
    #img.show()
    #print(f)
    img=img.save(filename)
    count+=1

100%|██████████| 319/319 [00:24<00:00, 12.85it/s]


In [ ]:
import cv2
import glob

In [ ]:
frameSize = (224, 224)
out = cv2.VideoWriter('/content/drive/MyDrive/ETVProject/ETV_Project/chiari_so.avi',cv2.VideoWriter_fourcc(*'MJPG'), 2, frameSize)

In [ ]:
test_images = natsorted(glob.glob("/content/yolov5/runs/detect/exp/*.png"))

In [ ]:
for filename in tqdm(test_images):
    img = cv2.imread(filename)
    img=cv2.resize(img, (224,224))
    out.write(img)


100%|██████████| 319/319 [00:03<00:00, 95.72it/s]


In [ ]:

out.release()

In [ ]:
print(phase1_idx, phase2_idx, phase3_idx, phase4_idx, phase5_idx, phase6_idx)

5 61 66 88 189 242
